In [ ]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, choose_dataloader
from ModelFunctions import multigpu_available, train, evaluate, inference

In [ ]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    
    atom_embeds = torch.cat(atom_features, 1)
    
    edge_index = system['edge_index_new'].long()

    edges_embeds = torch.cat(edge_features, 1)
    
    return Data(x=atom_embeds.to(device), edge_index=edge_index.to(device), edge_attr=edges_embeds.to(device))

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)
$$

Гамма лежит в апдейт, квадратик в aggr, а фи в месседж; в этом примере квадратик -- суммирование

In [ ]:
class GConv(MessagePassing):
    def __init__(self):
        super(GConv, self).__init__(aggr='add')  # "Add" aggregation

    def forward(self, batch):
        x = batch['x']
        edge_index = batch['edge_index']
        edge_attr = batch['edge_attr']
        
        # x has shape [N -- количество атомов в системе(батче), in_channels -- размерность вектора-атома]
        # edge_index has shape [2, E] -- каждое ребро задаётся парой вершин

        # Start propagating messages. 
    
        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)

    def message(self, x, x_i, x_j, edge_attr):
        # your function
        pass
        
    def update(self, aggr_out, x, edge_attr, edge_index):        
        #your function
        pass

In [ ]:
class ConvNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()          
        self.conv = GConv()
        
    def forward(self, batch):
        #your function
        pass

In [ ]:
#config
batch_size = 2
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 20

In [ ]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [ ]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

In [ ]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/10k/train/data.lmdb")

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = choose_dataloader(training_set, batch_size=batch_size)

In [ ]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data.lmdb")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = choose_dataloader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [ ]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

In [ ]:
#model
model = ConvNN(dim_atom=training_set[0][0]['x'].shape[1], dim_edge=training_set[0][0]['edge_attr'].shape[1])
if multigpu_available():
    model = DataParallel(model)

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)

In [ ]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [ ]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr
}


#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

## Training

In [ ]:
%%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))